In [32]:
import pandas as pd
data = pd.read_csv('dataset_new.csv')
data.head()

,user_id,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,99sTYU0wpK5yKwegir5Occ,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,NbjaXVDGeKYpqaj5dNqwGb,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
2,BoeuXIhPugS9sZoM6WATB9,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
3,VtxegG3W6k4gaMmTMxJQWa,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
4,0gLR9GYKEr0fY0Yu51tgU3,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic


In [33]:
import re
data=data.dropna()
data.shape


(305245, 22)

In [34]:
data = data.drop('Unnamed: 0', axis=1)
data.head()

,user_id,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,99sTYU0wpK5yKwegir5Occ,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
1,NbjaXVDGeKYpqaj5dNqwGb,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
2,BoeuXIhPugS9sZoM6WATB9,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
3,VtxegG3W6k4gaMmTMxJQWa,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic
4,0gLR9GYKEr0fY0Yu51tgU3,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,...,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic


In [35]:
from gensim.models import Word2Vec

In [36]:
data['artists'] = data['artists'].apply(lambda x: x.split(';'))

In [37]:
user_artist_sequences = data.groupby('user_id')['artists'].apply(list).tolist()

In [38]:
#sublist oluşturma
user_artist_sequences = [[artist for sublist in user_artists for artist in sublist] for user_artists in user_artist_sequences]
model = Word2Vec(user_artist_sequences, vector_size=50, window=5, min_count=1, sg=1)
print(model.wv.most_similar('Gen Hoshino'))

[('Motohiro Hata', 0.8962546586990356), ('KT Tunstall', 0.8915998935699463), ('Hanare Gumi', 0.8717014193534851), ('Gabrielle Aplin', 0.8649213910102844), ('Long Shot Party', 0.8579398989677429), ('Dancing Mood', 0.8537013530731201), ('Sara Bareilles', 0.8474150896072388), ('Tamio Okuda', 0.8422428369522095), ('Abdon Mech', 0.8400031924247742), ('Hikaru Utada', 0.8324142694473267)]


In [39]:
artist_vectors = {artist: model.wv[artist] for artist in model.wv.index_to_key}
vectors_df = pd.DataFrame.from_dict(artist_vectors, orient='index')

data['artist_vector'] = data['artists'].apply(lambda artists: [artist_vectors[artist] for artist in artists])



In [40]:
data.head()

,user_id,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,artist_vector
0,99sTYU0wpK5yKwegir5Occ,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,73,230666,False,0.676,0.461,...,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic,"[[-0.6215569, 0.031457033, 0.27063045, 0.32381..."
1,NbjaXVDGeKYpqaj5dNqwGb,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,73,230666,False,0.676,0.461,...,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic,"[[-0.6215569, 0.031457033, 0.27063045, 0.32381..."
2,BoeuXIhPugS9sZoM6WATB9,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,73,230666,False,0.676,0.461,...,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic,"[[-0.6215569, 0.031457033, 0.27063045, 0.32381..."
3,VtxegG3W6k4gaMmTMxJQWa,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,73,230666,False,0.676,0.461,...,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic,"[[-0.6215569, 0.031457033, 0.27063045, 0.32381..."
4,0gLR9GYKEr0fY0Yu51tgU3,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,73,230666,False,0.676,0.461,...,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic,"[[-0.6215569, 0.031457033, 0.27063045, 0.32381..."


In [41]:
sentences = [track.split() for track in data['track_name']]

# Word2Vec modelini oluşturma
model = Word2Vec(sentences, vector_size=10, window=2, min_count=1, workers=4)

# Her track için embedding vektörlerini alma
embedding_dict = {}
for track in data['track_name'].unique():
    words = track.split()
    track_vector = sum(model.wv[word] for word in words if word in model.wv) / len(words)
    embedding_dict[track] = track_vector



# Sonuçları DataFrame'e ekleme
embedding_df = pd.DataFrame.from_dict(embedding_dict, orient='index')


threshold = embedding_df.mean().mean()  # Tüm DataFrame'in ortalaması

# İkili normalizasyon
binary_normalized_df = (embedding_df > threshold).astype(int)

# Sonuçları gösterme
print(binary_normalized_df)

                            0  1  2  3  4  5  6  7  8  9
Comedy                      0  0  0  0  0  1  0  0  0  0
Ghost - Acoustic            1  1  1  0  1  0  1  1  0  0
To Begin Again              1  1  1  0  1  0  1  1  0  0
Can't Help Falling In Love  0  1  1  0  1  0  1  1  0  0
Hold On                     0  1  1  1  1  0  0  1  0  0
...                        .. .. .. .. .. .. .. .. .. ..
Frecuencia Álmica, Pt. 4    0  0  1  0  0  0  1  1  0  0
Sleep My Little Boy         0  1  1  0  1  0  1  1  0  0
Water Into Light            0  1  1  0  1  0  0  1  0  0
Miss Perfumado              0  1  1  0  1  0  1  1  0  1
Barbincor                   1  0  1  0  0  1  0  0  0  0

[73608 rows x 10 columns]
